# Evaluate classification models & dump images for diagnosis/labelling

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib notebook
# %matplotlib inline

In [2]:
import sys

from catboost import CatBoostClassifier, Pool
from scipy.ndimage import binary_dilation
from sklearn import clone
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from pathlib import Path
import numpy as np
import pandas as pd
from definitions import ROOT_DIR
import matplotlib.pyplot as plt
import seaborn as sns
from metaspace.sm_annotation_utils import SMInstance
from metaspace.image_processing import clip_hotspots

import getpass
from metaspace import SMInstance
from datetime import datetime

In [3]:
# Suppress warnings, because many models spam them during feature selection
# as some subsets of features just don't have enough information to make
# a good model.
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

## Utility functions

In [4]:
def colorize_image_with_mask(image, mask):
    """Plotting function for combining a colorized ion image with a spot mask"""
    image = clip_hotspots(image)
    on_spot_colorized = plt.cm.cividis(image)
    off_spot_colorized = plt.cm.magma(image)
    return np.where(mask[:,:,np.newaxis], on_spot_colorized, off_spot_colorized)
    
def save_image_with_mask(image, mask, fname):
    plt.imsave(fname, colorize_image_with_mask(image, mask))

In [5]:
def crop_zeros(img):
    """Crop an image, removing all empty outer rows/columns"""
    cols = np.flatnonzero(np.count_nonzero(img, axis=0) != 0)
    rows = np.flatnonzero(np.count_nonzero(img, axis=1) != 0)
    top = rows[0]
    bottom = rows[-1] + 1
    left = cols[0]
    right = cols[-1] + 1

    return img[top:bottom, left:right]

In [6]:
def get_mispredictions(model, X, y):
    """
    Find which values would be mispredicted, returning two lists:
        * indexes of items that would be falsely predicted as positives
        * indexes of items that would be falsely predicted as negatives
        
    cross_val_predict uses a shuffled 5-fold test-train split so that each chunk of 
    20% of the input data gets its own model that was trained on the other 80%, 
    ensuring that the items being predicted aren't included in the training data.
    """
    preds = cross_val_predict(model, X, y)
    mispreds = preds != y
    fpos_idxs = np.flatnonzero(mispreds & ~y)
    fneg_idxs = np.flatnonzero(mispreds & y)
        
    return fpos_idxs, fneg_idxs

## Paths

In [7]:
p_root_dir = Path(ROOT_DIR)
# p_grids = p_root_dir / "2_grid_calibration/grid_masks"
# p_wellmap = p_root_dir / "3_metric_extraction/Well_2_compound.csv"
# p_stats = p_root_dir / "4_metric_extension"
# p_analysis = p_root_dir  / "5_well_classification"
# p_labelled_set = p_analysis / "labelled_set"
# p_datasets = p_labelled_set / "Datasets.csv"

p_analysis = p_root_dir  / "4_model_evaluation"
p_grids = p_analysis / r"data_for_model_training\labelled_set_masks\grid_masks"
p_labelled_set = p_analysis / r"data_for_model_training\labelled_set"
p_wellmap = p_root_dir / "5_data_analysis/wellmap.csv"
p_metrics = p_analysis / "metrics.csv"

# Paths for evaluation
p_eval = p_analysis/ "model_evaluation"
p_eval_preds = p_eval / 'predictions.csv'
# False positives/negatives - preview output from model prediction for molecules with known labels
# Note that all files in these directories are cleared before a prediction run
p_eval_fpos = p_eval / 'false_positives'
p_eval_fneg = p_eval / 'false_negatives'
p_eval_tpos = p_eval / 'true_positives'
p_eval_tneg = p_eval / 'true_negatives'
# Unknown positives/negatives - preview output from model prediction for molecules with no label
# Note that all files in these directories are cleared before a prediction run
p_eval_upos = p_eval / 'unknown_positives'
p_eval_uneg = p_eval / 'unknown_negatives'
# Manually labeled positives/negatives - Move preview files from any of the above directories into 
# these directories to add to the labelled data. Make sure to re-run the appropriate steps 
# in "Input data" to detect the changes
p_eval_lpos = p_eval / 'manual_label_positives'
p_eval_lneg = p_eval / 'manual_label_negatives'

# Directories for three-state positive/unsure/negative classification
p_tri_pos = p_eval / 'three-state' / 'positive'
p_tri_unk = p_eval / 'three-state' / 'unsure'
p_tri_neg = p_eval / 'three-state' / 'negative'


# METASPACE
database = ('Spotting_project_compounds-v9', 'feb2021')
fdr = 0.5

In [8]:
# Log into metaspace
sm = SMInstance(host='https://metaspace2020.eu')

if not sm.logged_in():
    # Using getpass here prevents the API key from being accidentally saved with this notebook.
    api_key = getpass.getpass(prompt='API key: ', stream=None)
    sm.login(api_key=api_key)

API key: ········


## Input data

In [9]:
# Get dataset IDs based on Quality_Labels.csv files 
dataset_ids = pd.concat([
    pd.read_csv(f)
    for f in p_labelled_set.rglob("*Quality_Labels.csv")
]).dataset_id.unique().tolist()


In [11]:
# Images from METASPACE
# NOTE: Hotspot clipping is applied at this step, so `np.max(image)` 
# represents the 99th percentile intensity for the rest of the script.
#
# Ignore any warnings about connection pools in this step

images = []
for i, ds_id in enumerate(dataset_ids):
    print(f'Downloading images for {ds_id} ({i}/{len(dataset_ids)})')
    for img in sm.dataset(id=ds_id).all_annotation_images(
        fdr=fdr, 
        database=database, 
        only_first_isotope=True, 
        scale_intensity=False, 
        hotspot_clipping=True
    ):
        # Exclude annotations with no first-isotopic-image
        if img[0] is not None:
            images.append({
                'dataset_id': ds_id,
                'formula': img.formula,
                'adduct': img.adduct,
                'neutral_loss': img.neutral_loss or '',
                'image': img[0],
                'filename': f'{ds_id}_{img.formula}_{img.adduct}_{img.neutral_loss}.png'.replace('+', ''),
            })

images_df = pd.DataFrame(images)

In [12]:
# Wellmap and grids
wellmap = pd.read_csv(p_wellmap)
grids = {
    ds_id: np.load(p_grids / f'{ds_id}.npy') 
    for ds_id in dataset_ids
}

In [13]:
# Image labels from Quality_Labels.csv files
labeled_anns = []
for i in p_labelled_set.rglob("*Quality_Labels.csv"):
    data = pd.read_csv(i)
    data = data.loc[:, ['dataset_id', 'formula', 'adduct', 'neutral_loss', 'score']]
    data.neutral_loss.fillna('', inplace=True)
    labeled_anns.append(data)

labeled_anns_df = pd.concat(labeled_anns)

### Import image labels from the manual_label directories

If you use these directories for labelling, re-run every cell from this point onwards

In [14]:
# Image labels from the "manual_label" directories
manual_labels = []
for score, labels_path in [(1, p_eval_lpos), (0, p_eval_lneg)]:
    labels_path.mkdir(parents=True, exist_ok=True)
    for f in labels_path.glob('*.png'):
        manual_labels.append({
            'filename': f.name,
            'manual_score': score,
        })
if manual_labels:
    manual_labels_df = pd.DataFrame(manual_labels)
else:
    manual_labels_df = pd.DataFrame({'filename': pd.Series(dtype=str), 'manual_score': pd.Series(dtype='i')})

In [16]:
# Combine them for easier access
merged_df = (
    images_df
    # Add `how=left` when merging with wellmap to include non-spotted formulas
    .merge(wellmap[['well', 'formula', 'name_short']], on=['formula'])
    .merge(labeled_anns_df, on=['dataset_id', 'formula', 'adduct', 'neutral_loss'], how='left')
    .merge(manual_labels_df, on='filename', how='left')
).reset_index()

# Merge the "manual_score" column into "score"
merged_df['score'] = np.where(merged_df.score.isna(), merged_df.manual_score, merged_df.score)
del merged_df['manual_score']

## Calculate metrics (or load pre-calculated)

In [17]:
#Calculate metrics
def calc_far_bg(mask, bg):
    """Gets mask for background pixels that are at least 4 radii away from the spot"""
    # 3 iterations = (1+3=)4x the spot radius
    expanded_spot = binary_dilation(mask, crop_zeros(mask), iterations=3)
    return bg & ~expanded_spot

def occ(px):
    """Calculates non-zero % of the given array"""
    return np.count_nonzero(px) / px.size


metrics = []
for row in merged_df.itertuples():
    grid = grids[row.dataset_id]
    
    mask = grid == row.well
    bg = grid == 0
    far_bg = calc_far_bg(mask, bg)
        
    in_mask = row.image[mask]
    in_bg = row.image[bg]
    in_far_bg = row.image[far_bg]
    in_other_spots = row.image[~bg & ~mask]
    
    # Calculate threshold (0.01 * 99th percentile) 
    # (note the image is already hotspot-removed, so the max is the 99th percentile)
    threshold = np.max(row.image) * 0.01

    metrics.append({
        # Original metrics
        # NOTE: The constant in the denominator of `on_off_ratio` was changed to
        # 0.001 as it seemed to produce slightly better results
        'occupancy_ratio': (occ(in_mask) * 100) / (occ(in_bg) * 100 + 1),
        'on_off_ratio': (np.mean(in_mask)) / (np.mean(in_bg) + 0.001),
        
        # Single-spot occupancy %
        'spot_occupancy': occ(in_mask),
        'spot_occupancy_thresholded': occ(in_mask > threshold),
        # Other occupancy metrics
        'image_occupancy': occ(row.image),
        'other_spots_occupancy': occ(in_other_spots),
        'bg_occupancy': occ(in_bg),
        'far_bg_occupancy': occ(in_bg),
        'occupancy_vs_far_bg_ratio' : (occ(in_mask) * 100) / (occ(in_far_bg) * 100 + 1),
        
        
        # How many spots have a non-zero pixel
        'in_n_spots': len(np.unique(grid[(grid != 0) & (row.image > threshold)])),
        
        # Intensity ratios
        'spot_intensity' : np.mean(in_mask),
        'intensity_vs_other_spots_ratio': np.mean(in_mask) / (np.mean(in_other_spots) + 0.001),
        'intensity_vs_far_bg_ratio': np.mean(in_mask) / (np.mean(in_far_bg) + 0.001),
        
        'score': row.score,
    })

metrics_df = pd.DataFrame(metrics, index=merged_df.index)
metrics_df.to_csv(p_metrics, index=False)
metrics_df.head()

,occupancy_ratio,on_off_ratio,spot_occupancy,spot_occupancy_thresholded,image_occupancy,other_spots_occupancy,bg_occupancy,far_bg_occupancy,occupancy_vs_far_bg_ratio,in_n_spots,spot_intensity,intensity_vs_other_spots_ratio,intensity_vs_far_bg_ratio,score
0,28.846154,55.776898,0.288462,0.230769,0.000487,0.000409,0.000000,0.000000,28.846154,2,0.055777,55.535919,55.776898,NaN
1,28.846154,89.592762,0.288462,0.269231,0.000487,0.000409,0.000000,0.000000,28.846154,4,0.089593,88.844338,89.592762,NaN
2,5.749543,27.731240,0.057692,0.057692,0.000102,0.000000,0.000034,0.000034,5.749220,1,0.027792,27.792176,27.730237,NaN
3,83.555635,244.620623,0.864865,0.810811,0.001634,0.002304,0.000351,0.000351,85.875339,2,0.249697,245.142977,249.578850,NaN
4,77.545055,198.311360,0.810811,0.810811,0.001465,0.001296,0.000456,0.000456,79.386183,4,0.201555,196.619429,201.086837,NaN


In [18]:
p_metrics

WindowsPath('d:/saharuka/spotting/20_matrices_git/spotting/analysis/20_matrices/4_model_evaluation/metrics.csv')

In [ ]:
# Or import pre-calculated metrics
metrics_df = pd.read_csv(p_metrics)
metrics_df['score'] = merged_df.score
metrics_df.head()

## Evaluate models

This section uses the calculated metrics and labeled data to train a set of models 
and find which features are best for predicting the labels. 
It uses two strategies for evaluation:

* Hold-out validation - this splits the labeled data into 80% for training, 20% for testing
* Cross-Validation - this uses the full labeled data, but trains 5 different models, each
    with a different combinations of inputs in the 80% training set, so that each input 
    can be tested by a model that didn't use that input as part of the training.
    This approach reports a much more numerically stable accuracy value it can use 
    the full input set for evaluation.
    However, it shouldn't be used for fine-tuning the model hyperparameters 
    (the input variables when constructing the model), as this can lead to overfitting.
    
   
The output is a DataFrame `eval_results_df` that shows for each model/# of features:
* Which combination of features worked best
* The accuracy/F1 scores
* The # of false positives & false negatives

In [19]:
# Prepare input data
input_df = metrics_df[~metrics_df.score.isna()]  # Exclude unlabeled rows
input_df = input_df.sample(frac=1.0)  # Shuffle rows
X = input_df.drop(columns=['score'])
y = input_df.score.astype('i').values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
# Models to try
models_to_eval = [
    CatBoostClassifier(verbose=False),
    LinearSVC(class_weight='balanced'),
    DecisionTreeClassifier(max_depth=3),
    BaggingClassifier(LinearSVC(), n_estimators=3, bootstrap_features=True),
]
max_features_to_consider = 5

eval_results = []

for model in models_to_eval:
    model_name = str(model)
    for n_features in range(1, max_features_to_consider + 1):
        print(model_name, n_features)
        # SequentialFeatureSelector finds the set of N features that give the best scores
        sfs = SequentialFeatureSelector(model, n_features_to_select=n_features, n_jobs=-1)
        sfs.fit(X_train, y_train)
        best_features = X.columns[sfs.support_]
        
        # Evaluate using cross-validation
        X_subset = X[best_features].values
        fpos_idxs, fneg_idxs = get_mispredictions(model, X_subset, y)
        # Use a repeating cross-validator so that results are averaged over ~50 runs
        cv = RepeatedStratifiedKFold()
        cv_scores = cross_validate(model, X_subset, y, cv=cv, scoring=['accuracy','f1'])
        cv_accuracy = np.mean(cv_scores['test_accuracy'])
        cv_f1 = np.mean(cv_scores['test_f1'])
        
        # Evaluate using hold-out validation
        trained_subset_model = clone(model).fit(X_train[best_features].values, y_train)
        holdout_accuracy = trained_subset_model.score(X_test[best_features].values, y_test)
        holdout_f1 = f1_score(y_test, trained_subset_model.predict(X_test[best_features].values))
        
        eval_results.append({
            'model': model_name,
            'n_features': n_features,
            'features': ', '.join(best_features),
            'cv_accuracy': cv_accuracy,
            'cv_f1': cv_f1,
            'holdout_accuracy': holdout_accuracy,
            'holdout_f1': holdout_f1,
            'n_fpos': len(fpos_idxs),
            'n_fneg': len(fneg_idxs),
            # Uncomment to include the idxs of false positives/negatives to see which
            # inputs are repeatedly mispredicted regardless of the model
            # 'fpos_idxs': fpos_idxs,
            # 'fneg_idxs': fneg_idxs,
        })
        
eval_results_df = pd.DataFrame(eval_results)

eval_results_df

<catboost.core.CatBoostClassifier object at 0x000001D4ABD7C490> 1
<catboost.core.CatBoostClassifier object at 0x000001D4ABD7C490> 2
<catboost.core.CatBoostClassifier object at 0x000001D4ABD7C490> 3
<catboost.core.CatBoostClassifier object at 0x000001D4ABD7C490> 4
<catboost.core.CatBoostClassifier object at 0x000001D4ABD7C490> 5


In [ ]:
# Show behavior of accuracy as number of features increases
sns.lineplot(data=eval_results_df, x='n_features', y='cv_accuracy', hue='model')

## Examine results for a specific model

In [ ]:
model = CatBoostClassifier(verbose=False)
# features = ['occupancy_ratio', 'intensity_vs_other_spots_ratio', 'spot_occupancy', 'spot_occupancy_thresholded'] #original
features = ['spot_occupancy_thresholded', 'occupancy_vs_far_bg_ratio', 'intensity_vs_far_bg_ratio', 'intensity_vs_other_spots_ratio']

### Option A: Train a new model

In [ ]:
# Train the model on labeled data
train_df = metrics_df[~metrics_df.score.isna()]  # Exclude unlabeled rows
train_df = input_df.sample(frac=1.0)  # Shuffle rows
X_df = train_df.drop(columns=['score'])[features]
y = train_df.score.astype('i').values
trained_model = clone(model).fit(X_df.values, y)

# Make predictions for unlabeled data
unlabeled_df = metrics_df[metrics_df.score.isna()][features]
unlabeled_predictions_df = pd.DataFrame({
    'pred_val': trained_model.predict_proba(unlabeled_df.values)[:, 1]
}, index=unlabeled_df.index)

# Make cross-validated predictions for labeled data
labeled_predictions_df = pd.DataFrame({
    'pred_val': cross_val_predict(model, X_df.values, y, method='predict_proba')[:, 1]
}, index=X_df.index)

# Combine predictions
predictions_df = pd.concat([unlabeled_predictions_df, labeled_predictions_df])

### Option B: Load an existing model
Uses a saved model from the last step of this file

NOTE: This approach doesn't use cross-validated predictions for the labelled training data,
so it shouldn't be used for analyzing the model or refining the training set.

In [ ]:
model.load_model(p_eval / 'model.json', format='json')

# Make predictions for all data
predictions_df = pd.DataFrame({
    'pred_val': model.predict_proba(metrics_df[features].values)[:, 1]
}, index=metrics_df.index)

### Both options: Assign labels to predictions

In [ ]:
# Make combined DF
output_df = merged_df.join(metrics_df.drop(columns='score')).join(predictions_df)

# Add two-state and three-state classes
output_df['pred_twostate'] = np.where(output_df.pred_val < 0.5, 0, 1)
unsure_range = [0.2, 0.8] # Lowest & highest values to include in the "unsure" class
# This assigns 0 = negative, 1 = unsure, 2 = positive
output_df['pred_threestate'] = np.digitize(output_df.pred_val, unsure_range)

### Write predictions CSV files

In [ ]:
timestamp = datetime.now().strftime("%d-%b-%Y") 
csv_df = output_df.drop(columns=['image', 'filename']) # Skip unwanted columns
csv_df.to_csv(p_root_dir / f"5_data_analysis/all_predictions_{timestamp}.csv")

for dataset_id, results_df in csv_df.groupby('dataset_id'):
    output_path = p_eval / f'{dataset_id}_predictions.csv'
    results_df.to_csv(output_path)

### Write image files into false positives, false negatives, etc.

In [ ]:
# Clean output directories
for output_path in [
    p_eval_fpos, p_eval_fneg, p_eval_tpos, p_eval_tneg, p_eval_upos, p_eval_uneg, 
    p_tri_pos, p_tri_unk, p_tri_neg
]:
    output_path.mkdir(parents=True, exist_ok=True)
    for f in output_path.glob('*.png'):
        f.unlink()  # Delete existing files

# Write images with two-state classification
for row in output_df.itertuples():
    mask = grids[row.dataset_id] == row.well
    
    # Figure out which directory to use
    if row.score == 0:
        twostate_path = [p_eval_tneg, p_eval_fpos][row.pred_twostate]
    elif row.score == 1:
        twostate_path = [p_eval_fneg, p_eval_tpos][row.pred_twostate]
    else:
        twostate_path = [p_eval_uneg, p_eval_upos][row.pred_twostate]
    
    save_image_with_mask(row.image, mask, twostate_path / row.filename)
    
# Write images with three-state classification
for row in output_df.itertuples():
    mask = grids[row.dataset_id] == row.well
    
    threestate_path = [p_tri_neg, p_tri_unk, p_tri_pos][row.pred_threestate]
    
    save_image_with_mask(row.image, mask, threestate_path / row.filename)

# Save trained model

Note: This JSON export only works for CatBoost. 
scikit-learn models don't have a standardized export format.

In [ ]:
trained_model.save_model(p_eval / 'model.json', format='json', pool=Pool(X_df.values, y))